In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

dosya_yolu="/content/data (1).csv"
data=pd.read_csv(dosya_yolu)

#ÖN İŞLEME
data.drop(["id","Unnamed: 32"],axis=1,inplace=True)
label_encoder=LabelEncoder()
data["diagnosis"]=label_encoder.fit_transform(data["diagnosis"])
X=data.drop("diagnosis",axis=1)
y=data["diagnosis"]
sc=StandardScaler()
X_scaled=sc.fit_transform(X)
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.3,random_state=42,stratify=y)

from sklearn.model_selection import GridSearchCV
# --- HİPERPARAMETRE OPTİMİZASYONU (GRID SEARCH) ---
svc=SVC()# Boş bir SVM modeli oluşturuyoruz.
# C: Hata toleransı. Küçük C = Çok hata yapar ama geneli iyi görür (Soft Margin).
# Büyük C = Hata yapmaktan korkar, veriyi ezberlemeye çalışır (Hard Margin).
param_grid={
    "C":[0.1,1,10,100,1000],
    # kernel: Veriyi nasıl böleceği. Düz çizgi mi (linear), eğri mi (rbf), polinom mu (poly)?
    "kernel":["linear","poly","rbf","sigmoid"],
    # degree: Polinomun derecesi (x^2, x^3...). Sadece 'poly' kernel için geçerlidir.
    'degree':[2,3,4],
    # gamma: Bir noktanın ne kadar uzağı etkileyeceği.
    # 'scale' ve 'auto' otomatik ayarlardır. Sayısal değer de verilebilir.
    "gamma":["scale","auto"],
    # coef0: Polinom ve Sigmoid kernel için serbest terim katsayısıdır. Modeli esnetir.
    "coef0":[0.0,0.1,0.5]
}
# GridSearchCV Kurulumu
# estimator=svc: Optimize edilecek model.
# param_grid: Denenecek ayarlar.
# cv=10: 10 Katlı çapraz doğrulama (Her ayarı 10 kez test et, sağlam olsun).
# n_jobs=-1: Bilgisayardaki TÜM işlemcileri kullan (Hız için çok önemli).
# verbose=1: İşlem sürerken ekrana bilgi bas.
grid_search=GridSearchCV(estimator=svc,
                         param_grid=param_grid,
                         scoring="accuracy",cv=10,n_jobs=-1,verbose=1)
# --- ARAMAYI BAŞLAT ---
# Bu işlem, yukarıdaki tüm kombinasyonları tek tek dener. Uzun sürebilir!
grid_search.fit(X_train,y_train)
# --- SONUÇLAR ---
# En iyi sonucu veren modeli ve ayarları alıyoruz.
best_model=grid_search.best_estimator_
best_params=grid_search.best_params_
print("En iyi parametreler:",best_params)

y_pred=best_model.predict(X_test)
accuracy=best_model.score(X_test,y_test)
print("En iyi modelin doğruluğu:",accuracy)

cm=confusion_matrix(y_test,y_pred)
cr=classification_report(y_test,y_pred,
                         target_names=["Benign","Malignant"])
print("Sınıflandırma Raporu:\n",cr)

plt.figure(figsize=(6,4))
sns.heatmap(cm,annot=True,fmt="d",cmap="Blues",xticklabels=["Benign","Malignant"],yticklabels=["Benign","Malignant"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest,f_classif
import matplotlib.pyplot as plt
import seaborn as sns

dosya_yolu="/content/data (1).csv"
data=pd.read_csv(dosya_yolu)

#ÖN İŞLEME
data.drop(["id","Unnamed: 32"],axis=1,inplace=True)
label_encoder=LabelEncoder()
data["diagnosis"]=label_encoder.fit_transform(data["diagnosis"])
X=data.drop("diagnosis",axis=1)
y=data["diagnosis"]
sc=StandardScaler()
X_scaled=sc.fit_transform(X)
# --- SMOTE (Synthetic Minority Over-sampling Technique) ---
# Azınlık sınıfı (Kanserli olanlar azsa) yapay verilerle çoğaltarak dengeyi sağlar.
smote=SMOTE(random_state=42)
# AÇIKLAMA (Senin Yorum Satırların):
# smote = SMOTE(sampling_strategy=0.5, random_state=42)
# ANLAMI: "Azınlık sınıfını (1), Çoğunluk sınıfının (0) yarısı kadar olana dek çoğalt."
# Örn: 300 tane 0 varsa, 1'leri artırıp 150 tane yapana kadar üretir.

# smote = SMOTE(sampling_strategy={1: 300}, random_state=42)
# ANLAMI: "1 numaralı sınıftan (Malignant) elimde tam 300 tane olana kadar üret."
# Bu manuel bir ayardır, sayıya direkt müdahale edersin.

#smote=SMOTE(sampling_strategy=0.5,random_state=42)
#smote=SMOTE(sampling_strategy={1: 300},random_state=42)

# fit_resample: Veriyi alıp, sentetik verileri ekleyip bize artırılmış (augmented) X ve y döndürür.
X_augmented,y_augmented=smote.fit_resample(X_scaled,y)

print(y.value_counts())
print(y_augmented.value_counts())
# --- ÖZELLİK SEÇİMİ (FEATURE SELECTION) ---
# SelectKBest: En iyi K tane özelliği seçer.
# score_func=f_classif: Özelliklerin hedefle ilişkisini ölçen istatistiksel test (ANOVA).
# k=10: 30 özellikten en etkili 10 tanesini seç diyoruz.
k_best_selector=SelectKBest(score_func=f_classif,k=10)
X_selected=k_best_selector.fit_transform(X_augmented,y_augmented)

#FİX :APPLY GET_SUPPORT() TO THE ORGİNAL FEATURE NAMES FROM X

# SEÇİLEN ÖZELLİKLERİN İSİMLERİNİ BULMA
# get_support(): Seçilen sütunlar için True, seçilmeyenler için False döndürür.
# X.columns[...] diyerek sadece True olanların (Seçilenlerin) isimlerini alıyoruz.
selected_feature=X.columns[k_best_selector.get_support()]
print("Seçilen Özellik: ")
print(selected_feature)
# --- EĞİTİM / TEST AYRIMI ---
# Artık elimizdeki veri hem dengelenmiş (augmented) hem de en iyi özellikleri seçilmiş (selected) veridir.
X_train,X_test,y_train,y_test=train_test_split(X_selected,y_augmented,test_size=0.2,random_state=42,stratify=y_augmented)

models={
    "Logistic Regression":LogisticRegression(),
    "Decision Tree":DecisionTreeClassifier(),
    "Random Forest":RandomForestClassifier(),
    "Gradient Boosting":GradientBoostingClassifier(),
    "AdaBoost":AdaBoostClassifier(),
    "SVM":SVC(),
    "KNN":KNeighborsClassifier(),
    "Naive Bayes":GaussianNB()
}
accuracies=[]
for model_name,model in models.items():
  model.fit(X_train,y_train)
  y_pred=model.predict(X_test)
  accuracy=accuracy_score(y_test,y_pred)
  accuracies.append((model_name,accuracy))
  print(f"{model_name} Accuracy: {accuracy: .4f}")

accuracies_df=pd.DataFrame(accuracies,columns=["Model","Accuracy"]).sort_values(by="Accuracy",ascending=False)
plt.figure(figsize=(10,6))
sns.barplot(x="Accuracy",y="Model",data=accuracies_df,palette="viridis",hue="Model",legend=False)
plt.xlabel("Accuracy")
plt.ylabel("Model")
plt.title("Model Doğrulukları")
plt.xticks(rotation=90)
plt.show()

